RECAP

Sieci Rekurencyjne posiadają pamięć o sekwencjach w danych wejściowych poprzez modeyfikację standardowej komórki perceptronu. To czyni je skuteczne w przetwarzaniu języka:

- skuteczność dla danych sekwencyjnych ze względu na pamięć w komórkach sieci

- komórki LSTM i GRU mają (ograniczoną) zdolność przetwarzania odległych zależności w sekwencji

- te dwie własności pozwalają na proste rozumienie kontekstu przez RNN, w szczególności LSTM lub GRU

SimpleRNN: prostsze i mają krótki kontekst

LSTM: bardziej skomplikowana komórka pamięci i możliwość skutecznego modelowania dłuższego kontekstu (dłuższe sekwencje)

In [ ]:
import sys
import numpy as np
import tensorflow as tf
import copy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Flatten, Dense, Dropout, GRU
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

### Stemming, Lemmatyzacja - procesowanie tekstu

**Stemming** usuwa końcówki fleksyjne, sprowadzając słowo do rdzenia (np. running → run). Jest szybki, ale mniej precyzyjny.


**Lemmatyzacja** używa słowników językowych, aby znaleźć poprawną formę podstawową (np. better → good). Jest dokładniejsza, ale wolniejsza.

Metody bywają przydatne w podnoszeniu dokładności modeli tekstowych do różnych zadań.

In [ ]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Pobieranie potrzebnych pakietów
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

In [ ]:
# Przykładowy tekst
text = "We love programming on a piece of paper."

# Tokenizacja wyrazów
words = word_tokenize(text)

In [ ]:
# Inicjalizacja Stemmera i Lemmatizera
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Stemming wyrazów
stemmed_words = [stemmer.stem(word) for word in words]

# Lematyzacja wyrazów
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

In [ ]:
print("Original Words:    ", words)
print("Stemmed Words:     ", stemmed_words)
print("Lemmatized Words:  ", lemmatized_words)

**TASK** Przetestować powyższe metody (stemming, lemmatzyacja) na wymyślonym przez siebie zdaniu.

### Reprezentacja tekstu w Sieci Neuronowej

Podział zdań na wyrazy - tokenizacja

In [ ]:
# Tokenizacja tekstu
text = "Never have I ever complained about SSN classes"
tokens = text.split(" ")
tokens

Kodowanie wyrazów na cyfry dla całego słownika - wszystkie słowa w zbiorze danych

In [ ]:
# Proste enkodowanie numeryczne
token_to_index = {word: idx + 1 for idx, word in enumerate(tokens)}
encoded_text = list(token_to_index.values())

Do przetwarzania zakodowanych słów przez sieć neuronową, wektory wejściowe (zwykle) muszą mieć ten sam rozmiar. W związku z tym, po kodowaniu wykonuje się przycinanie lub dodawanie augmentowanych wartości do próbek wejściowych.

**TASK** W poniższej komórce uzupełnić wartość `maxlen` funkcji `pad_sequences` na długość pierwszego wektora wejściowego z listy (`sequences`). Opisać co dzieje się podczas paddingu dla różnych wektorów wejściowych z tej listy.

In [ ]:
# Padding - wypełnianie zaenkodowanych sekwencji żeby były jednej długości
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = [[1, 2, 3, 4, 5, 6, 7, 8],[1, 1, 1, 2, 3, 4, 5, 6, 7, 8],  [1, 2, 3, 4, 5, 6]]
padded_sequences = pad_sequences(sequences, maxlen=<PUT_VALUE_HERE>, padding="post")
print(padded_sequences)


### Bag of words model


Model Bag of Words (BoW) to jedna z najprostszych metod reprezentacji tekstu w formie numerycznej. W BoW każdy dokument jest zamieniany na wektor o wymiarach odpowiadających liczbie unikalnych słów w korpusie, gdzie wartości reprezentują liczbę wystąpień danego słowa w dokumencie. Model ten ignoruje kolejność słów i zależności między nimi, co sprawia, że nie uchwyca kontekstu, ale jest efektywny i często stosowany w klasyfikacji tekstu oraz wyszukiwaniu informacji. Udoskonaloną wersją BoW jest podejście TF-IDF, które uwzględnia częstotliwość słów w dokumentach, aby lepiej różnicować istotne terminy.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Przykładowe zdania
corpus = [
    "I love AGH",
    "The longer I think about it the less I understand it",
    "It is better to walk slowly in a good diirection than run around a false hope"
]

# Inicjalizacja COunt Vectorizera
vectorizer = CountVectorizer()

# Fit transform
X = vectorizer.fit_transform(corpus)

print("Nazwy cech", vectorizer.get_feature_names_out())
print("Reprezentacja BoW:\n", X.toarray())


**TASK** Wypróbować powyższe modele na zdaniu "The longer I study at AGH the less I understand it, but I trust the process". Wytłumaczyć dlaczego wektor BoW przyjął takie wartości - interpretacja działania modelu.

###  TF-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) to miara wykorzystywana w przetwarzaniu języka naturalnego, która służy do oceny ważności słów w dokumencie w kontekście zbioru dokumentów.

Składa się z dwóch głównych komponentów:

- Term Frequency (TF): Jest to miara częstotliwości wystąpienia danego terminu (słowa) w dokumencie. Oblicza się ją jako stosunek liczby wystąpień słowa do całkowitej liczby słów w dokumencie:
  - liczba wystąpień terminu t w dokumencie, dzielona przez
  - całkowita liczba słów w dokumencie

- Inverse Document Frequency (IDF): Jest to miara rzadkości danego terminu w całym zbiorze dokumentów. Oblicza się ją na podstawie liczby dokumentów, w których występuje dany termin. Jeśli termin występuje w wielu dokumentach, to jego waga będzie niższa. IDF oblicza się jako logarytm odwrotności proporcji dokumentów zawierających dany termin do całkowitej liczby dokumentów:
   - IDF(t)=log⁡(N/df(t))

    Gdzie:
    - N to całkowita liczba dokumentów,

    - df(t) to liczba dokumentów zawierających termin t.

Finalnie, TF-IDF to po prostu iloczyn obu tych miar:

TF-IDF(t)=TF(t)×IDF(t)

Celem TF-IDF jest nadanie większej wagi tym słowom, które są istotne w danym dokumencie, ale rzadko występują w innych dokumentach w zbiorze. Dzięki temu można lepiej ocenić, które terminy najlepiej opisują treść dokumentu, a które są mniej istotne lub powszechne.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Przykładowe dokumenty
corpus = [
    "Zagrać pauzą, ciszą, akcentem, zmianą rytmu mówienia. Tego wciąż nikt nie uczy",
    "Fizycznej gotówki mamy nieprzebrane ilości",
    "Płacimy dobrze, ale wymagamy też bardzo dużo",
    "Nie da się przeszczepić organu od martwego dawcy"
]

# Tworzenie modelu TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

# Wypisanie wyników
print("Słowa w słowniku:", vectorizer.get_feature_names_out())
print("Macierz TF-IDF:\n", X.toarray())


Tradycyjne metody, takie jak kodowanie numeryczne, Bag of Words (BoW) czy TF-IDF, reprezentują tekst w sposób numeryczny, ale słąbo uwzględniają znaczenia słów ani ich kontekstu. Słowa, które są synonimami lub mają podobne znaczenie, w tych podejściach są traktowane jako niezależne jednostki.

**Word embeddings** to reprezentacje wektorowe słów, kodując ich znaczenie w przestrzeni numerycznej. Są to gęste, ciągłe wektory o stałej długości, które pozwalają na uchwycenie semantycznych i syntaktycznych relacji między słowami.
Podczas używania Word Embeddings, słowa o podobnym znaczeniu znajdują się blisko siebie w przestrzeni wektorowej. Modele Sieci Neuronowych uczą się mapować słowa na wektory poprzez analizę ich kontekstu w dużych zbiorach danych tekstowych, np. `Word2Vec` lub transformery.


Przykładowe architektury modeli Word Embedding:

CBOW (Continuous Bag of Words): Model przwiduje słowo na podstawie podanych wyrazów z otoczenia przewidywanego słowa.

Example:

- Input: ["The", "longer", "I", "about", "it"]
- Label: "think"

Skip-gram: Model przewiduje sąsiednie wyrazy dla wybranego słowa wejściowego.

Example:
- Input: "think"
- Label: ["The", "longer", "I", "about", "it"]

Poniżej znajduje się przykładowa implementacja modelu Word2Vec w Keras. Dane są augmentowane dla ułatwienia implementacji.

In [ ]:
# Defninicja danych testowych
corpus = [
    "I love AGH",
    "The longer I think about it the less I understand it",
    "It is better to walk slowly in a good direction than run around a false hope"
]

input = [
    "I love",
    "The longer I think about it the I understand it",
    "It is better to walk slowly in a good direction than run around a hope"
]

label = [
    "AGH",
    "less",
    "false"
]

# Przygotowanie danych do modelu

# Tokenizacja
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

# Definicja rozmiaru wektora wejsciwego - ilość słów w słowniku
vocab_size = len(tokenizer.word_index) + 1

# Przygotowanie sekwencji
train_sequences = tokenizer.texts_to_sequences(input)
train_padded_sequences = np.array(pad_sequences(train_sequences, maxlen=vocab_size, padding='post'))

# Przygotowanie sekwencji
label_sequences = np.array(tokenizer.texts_to_sequences(label))


In [ ]:
print(train_padded_sequences)
print(label_sequences)

In [ ]:
# Definicja modelu WordEmbedding
embedding_dim = 8  # Define embedding size
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=train_padded_sequences.shape[1]),
    Flatten(),
    Dense(8, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

# Kompilacja modelu
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
# Trening modelu na zaugmentowanych danych
model.fit(train_padded_sequences, label_sequences.flatten(), epochs=10, verbose=1)


In [ ]:
# Ekstrakcja embeddingów dla poszczególnych słow modelu
embedding_layer = model.layers[0]
embedding_matrix = embedding_layer.get_weights()[0]

In [ ]:
embedding_matrix.shape

In [ ]:
# Wyciągnięcie embeddingów dla poszczególnych słow
word_embeddings = {word: embedding_matrix[idx] for word, idx in tokenizer.word_index.items()}

# Przykład użycia
word = "love"
if word in word_embeddings:
    print(f"Vector for word '{word}':")
    print(word_embeddings[word])
else:
    print(f"Word '{word}' not in vocabulary")

**TASK** Przetestować powyższy model enkodujący na wybranym przez siebie zdaniu. Należy zdefiniować zdanie, wykonać tokenizacje, padding i wywołać predykcje modelu. Opisać dlaczego predykcja modelu ma taki rozmiar.

### Analiza sentymentu z Siecią LSTM dla zbiory IMBD film reviews

**TIP** Warto zmienić środowisko programistyczne na środowisko z *GPU*.

Przygotowanie danych do treningu

In [ ]:
# Load IMDB dataset
max_features = 10000
max_len = 200
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)


**TASK** Zdefiniować sieć Simple RNN. Dodać warsty: `Embedding` i `SimpleRNN`.

In [ ]:
rnn_model = Sequential([
    <PLACEHOLDER_FOR_EMBEDDING_LAYER>,
    <PLACEHOLDER_FOR_SIMPLERNN_LAYER>,
    Dense(1, activation='sigmoid')
])

rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Trening i ewaluacja modelu SimpleRNN
rnn_model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))
rnn_acc = rnn_model.evaluate(x_test, y_test, verbose=0)[1]

**TASK** Zdefiniowac sieć LSTM do predykcji sentymentu zbioru danych IMBD. Wytrenować model LSTM podobnie jak model SimpleRNN powyżej. Wyznaczyć metrykę accuracy z treningu.

In [ ]:
lstm_model = Sequential()

In [ ]:
# Trening i ewaluacja modelu LSTM
lstm_model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))
lstm_acc = lstm_model.evaluate(x_test, y_test, verbose=0)[1]

Przygotowanie tekstu z tokenów dla porównania dokładności klasyfikacji RNN z pretrenowanym transformerem.

In [ ]:
# Pobranie i stworzenie mapy dla zbioru danych imbd
word_index = imdb.get_word_index()

# Odwrocenie word_index na tekst
index_word = {v + 3: k for k, v in word_index.items()}  # Offset by 3 (IMDB special tokens)
index_word[0], index_word[1], index_word[2], index_word[3] = "<PAD>", "<START>", "<UNK>", "<UNUSED>"

In [ ]:
def decode_review(encoded_review):
    return " ".join([index_word.get(i, "?") for i in encoded_review])

In [ ]:
# Dekodowanie tokenow
x_test_texts = [decode_review(seq) for seq in x_test]

Przykładowy zdekodowany tekst

In [ ]:
x_test_texts[1]

Ładowanie transformera z biblioteki hugging face

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")

In [ ]:
# Przykład predykcji
sentiment_pipeline(x_test_texts[1])

In [ ]:
# Mapa do etykiet dla transformera
map_label = {"NEGATIVE": 0, "POSITIVE": 1}

Predykcje z użyciem transformera.

**NOTE**: Predykcje mogą chwile potrwać.

In [ ]:
pred_transformer = sentiment_pipeline(x_test_texts, batch_size=100, truncation=True)

In [ ]:
pred_trans_mapped = [map_label[item["label"]] for item in pred_transformer]

Obliczenie accuracy dla transformera

In [ ]:
transformer_acc = accuracy_score(y_test, pred_trans_mapped)

In [ ]:
# Comparison results
print(f"RNN Accuracy: {rnn_acc:.4f}")
print(f"LSTM Accuracy: {lstm_acc:.4f}")
print(f"Transformer Accuracy: {transformer_acc:.4f}")


**TASK** Wykonać predykcję dla 1000 losowych danych z zbioru danych IMBD z użyciem SimpleRNN, LSTM i transformera (`sentiment_pipeline`). Wyznaczyć metryki accuracy, precision i recall dla każdego modelu.

**TASK** Podzielić dataset IMBD na dane treningowe, walidacyjne i testowe. Powtórzyć definicję i trening modeli SimpleRNN i LSTM. Wytrenować oba modele RNN na danych treningowych i użyć danych walidacyjnych do celów walidacji podczas treningu. Następnie porównać dokładność modeli SimpleRNN, LSTM i transformer (`sentiment_pipeline`) na danych testowych, które nie były używane do treningu modelu ani walidacji podczas treningu.

## Generacja tekstu

**NOTE** Należy dodać plik `wonderland.txt` z folderu Lab4 w repozytorium do plików aktywnego środowiska Google Colab.

Ładowanie danych

In [ ]:
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [ ]:
# Stworzenie mapy znaków do int
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Przygotowanie danych wejściowych do sieci:
- podział danych na X i y, które reprezentują tekst i jego kontynuację, której generowanie będziemy trenować
- zmiana znaków tekstowych na liczby

In [ ]:
seq_length = 100
x_enc = []
y_enc = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    x_enc.append([char_to_int[char] for char in seq_in])
    y_enc.append(char_to_int[seq_out])
n_patterns = len(x_enc)
print("Total Patterns: ", n_patterns)

In [ ]:
x_enc[3][:5]

In [ ]:
y_enc[3]

In [ ]:
# reshape X do sieci [samples, time steps, features]
X = np.reshape(x_enc, (n_patterns, seq_length, 1))
# Normalizacja danych
X = X / float(n_vocab)
# kodowanie targetu
y = to_categorical(y_enc)

In [ ]:
# Definicja modelu
model = Sequential([
    LSTM(128, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.1),
    Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# stworzenie mapy int to char - odwrócenie działania modelu
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# definicja checkpointu modelu
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)
callbacks_list = [checkpoint]

**TIP** Przy wielokrotnym trenowaniu modelu można wyłączyć tworzenie callbacku, żeby nie powielać zapisywania modeli.

In [ ]:
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)

In [ ]:
# generacja tekstu z losowym początkiem
start = np.random.randint(0, len(x_enc)-1)
pattern = x_enc[start]
start_pattern = copy.copy(pattern)
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

In [ ]:
# generacja tekstu przez n iteracji
n = 25
generated_text = []

for i in range(n):
    x = np.reshape(pattern, (1, len(pattern), 1))
    # normalizacja
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    # Wybranie następnego znaku zgodnie z największym prawdopodobieństwem zwróconym przez model
    index = np.argmax(prediction)
    # konwersja predykcji do znaku
    result = int_to_char[index]
    pattern.append(index)
    generated_text.append(result)
    # aktualizacja danych wejściowych o wygenerowany znak - podobnie jak w predykcji dla szeregów czasowych
    pattern = pattern[1:len(pattern)]

Wygenerowany text

In [ ]:
print(("").join([item for item in generated_text]))

Połączony wygenerowany tekst z wzorem początkowym

In [ ]:
start_pattern_text = [int_to_char[p] for p in start_pattern]
print(("").join([item for item in [*start_pattern_text, *generated_text]]))

**TASK** Próba poprawy generowanego tekstu (jako, że nie mamy etykiet, miara poprawy jest bardziej analityczna niż numeryczna):
- zwiększyć model LSTM (więcej warstw, dropout itd.)
- zamienić komórki LSTM na GRU (https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU)
- usprawnić parametry treningu: rozmiar batcha, ilość epok
- zmienić ilość przewidywanych znaków przez sieć
- zmienić ilość słow wejściowych w sieci
- usunąć niepotrzebne znaki z tekstu (text-processing)
- użyć paddingu do tworzenia sekwencji
- zmienić sposób wybierania następnego znaku przez model. Początkowo jest znak z największym prawdopodobieństwem. Można to zmienić na ważone losowe dobieranie jednego z `n` znaków z największym prawdopodobieństwem z predykcji.
- na koniec sprawdzić i podać ilość parametrów trenowanlych w modelu

**TASK** ($$$) Zaimplementować klasę z komórką LSTM wg. oryginalnej implementacji z publikacji: https://doi.org/10.1162/neco.1997.9.8.1735. Pomocnicze materiały: (https://www.geeksforgeeks.org/deep-learning-introduction-to-long-short-term-memory/, https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Komórka powinna być zaimplentowana tylko do propagowania informacji w przód i przetestowana na sztucznie wygenerowanych wartościach wag i próbek.